In [2]:
import numpy as np
import pandas as pd
import os
import math
from datetime import datetime
from dateutil import parser

In [3]:
logger = pd.read_csv('Truck Logger Data/Truck 1 Data Logger.csv')
logger['timestamp'] = pd.to_datetime(logger.Date + " " + logger.Time)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (33,45,65,84,88,91,92,94,110,116,122,124,130,134,135,136,156) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def clean_event_date_time(event):
    real_time = event.OCUR_TS[0:8]
    pm_or_am = event.OCUR_TS[-2:]
    
    cleaned_time = real_time + pm_or_am
    cleaned_time = cleaned_time.replace(".", ":")
    
    cleaned_date = event.OCUR_TS_Date
    
    cleaned_date_time = cleaned_date + " " + cleaned_time

    return parser.parse(cleaned_date_time)

In [5]:
op_events = pd.read_csv('Truck Events Files/operator event ver 1.csv', encoding='cp1252')
op_events_clean = op_events.dropna()
op_events_clean['timestamp'] = op_events_clean.apply(clean_event_date_time, axis=1)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [6]:
payloads = pd.read_csv(
    'Payload files/Payload trucks 1_10 updated.csv', encoding='cp1252')
payloads = payloads[payloads['MACH_SER_NO'] =='Truck 1']

/home/ubuntu/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0,1,2,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
def round_payload(event):
    payload = event['Truck Payload']
    return round(math.floor(payload*10)/10, 1)

In [8]:
logger_round = logger[logger['Truck Payload'] != 'Parameter not available']
# logger_round['Truck Payload'].unique()
logger_round['PAYLD_WT'] = logger_round.apply(round_payload, axis=1) 

TypeError: ('a float is required', 'occurred at index 0')

In [ ]:
logger_test = logger_round[logger_round['PAYLD_WT'].isin(payloads['PAYLD_WT'])]

In [ ]:
def new_date(event):
    hold = event['OCUR_TS Date']
    date = datetime.strptime(hold, '%d-%b-%y')
    out = datetime.strftime(date, '%m/%d/%Y')
    return out

In [ ]:
payloads['Date'] = payloads.apply(new_date, axis=1)

In [ ]:
#logger_test contains all entries in the logger where the rounded payload weight is in the payloads data frame
# logger_test.merge(payloads, on=['PAYLD_WT'])